In [55]:
import torch
import torch.nn as nn

## Define Model

In [56]:
class Reshape(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return x.view(-1, x.numel() / x.size(0))
    
    def __repr__(self):
        return "Reshaper"

In [57]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            Reshape(),
            nn.Dropout(p=.25),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(inplace=True),
            nn.Dropout(p=.25),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Dropout(p=.25),
            nn.Linear(84, 10)
        )
              
    def forward(self, x):
        return self.model(x)
        

### Make sure model work as expected

In [58]:
model = Net()
model

Net(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Reshaper
    (9): Dropout(p=0.25)
    (10): Linear(in_features=400, out_features=120, bias=True)
    (11): ReLU(inplace)
    (12): Dropout(p=0.25)
    (13): Linear(in_features=120, out_features=84, bias=True)
    (14): ReLU(inplace)
    (15): Dropout(p=0.25)
    (16): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [59]:
x = torch.rand(2, 1, 32, 32)
out = model(x)
out.size()

torch.Size([2, 10])

## Define Loss Function

In [60]:
ce_loss = nn.CrossEntropyLoss()
# loss = ce_loss(output, target)

## Define Optimizer

In [74]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Import Dataset

In [75]:
import torchvision
import torchvision.transforms as transforms

In [76]:
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# transform = transforms.ToTensor()

In [77]:
mnist_train = torchvision.datasets.MNIST('.', train=True, download=True, transform=transform)

In [78]:
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: .
    Transforms (if any): Compose(
                             Resize(size=32, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

In [79]:
trainloader = torch.utils.data.DataLoader(mnist_train, batch_size=128,
                                          shuffle=True, num_workers=4)
dataiter = iter(trainloader)
images, labels = dataiter.next()

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x111783fd0>>
Traceback (most recent call last):
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/Users/leemeng/anaconda3/envs/p04/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 86, in rebuild_storage_filename
    storage = cls._new_shared_filename(manager, handle, size)
RuntimeError: Interrupted system call at /Users/leemeng/Documents/Code/pytorch/torch/lib/libshm/core.cpp:125


In [80]:
len(trainloader) * 32

15008

## Summary writer

In [81]:
from tensorboardX import SummaryWriter
sw = SummaryWriter(log_dir='./runs')

## Train Model

In [82]:
%%time
for epoch in range(4):
    running_loss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = ce_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    running_loss /= len(trainloader)
    print(running_loss)
    sw.add_scalar('Loss', running_loss, epoch)


0.11734940768924476
0.08565971892732166
0.07260072700131232
0.06380086491750216
CPU times: user 9min 56s, sys: 1min 18s, total: 11min 15s
Wall time: 1min 52s


## Evaluation

In [83]:
mnist_test = torchvision.datasets.MNIST('.', train=False, download=True, transform=transform)

In [84]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: .
    Transforms (if any): Compose(
                             Resize(size=32, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

In [85]:
testloader = torch.utils.data.DataLoader(mnist_test, batch_size=128, num_workers=2)

In [86]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
model.train()



Accuracy of the network on the 10000 test images: 98 %


Net(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Reshaper
    (9): Dropout(p=0.25)
    (10): Linear(in_features=400, out_features=120, bias=True)
    (11): ReLU(inplace)
    (12): Dropout(p=0.25)
    (13): Linear(in_features=120, out_features=84, bias=True)
    (14): ReLU(inplace)
    (15): Dropout(p=0.25)
    (16): Linear(in_features=84, out_features=10, bias=True)
  )
)